In [38]:
import numpy as np
import os
import pandas as pd
import re

In [ ]:
#
# Data exploration
# > Still need to include data on TK auto-promotions
# > Need to include data from missing years
#
data_directory = '../cleaned/'
file_columns = dict()

for filename in os.listdir(data_directory):
    if filename.endswith('.csv'):
        file_columns[filename] = pd.read_csv(os.path.join(data_directory, filename)).columns.tolist()

for filename, columns in file_columns.items():
    print(f"{filename}: {columns}")

In [140]:
file_descriptions = [
    ( '25-26', '3. March 2025 SFUSD Requests by School.csv', 'idSchool', 'SchoolName', 'Program code', 'Program Name', 'Grade' ),
    ( '24-25', '3. March 2024 SFUSD Requests by school.xlsx - Table 1.csv', 'idSchool', 'SchoolName', 'Program Code', 'Program Name', 'Grade' ),
    ( '23-24', '2023-24 Main Round Requests by School-Grade-Program_1.xlsx - Table 1.csv', 'School ID', 'School Name', 'Program', 'Program Name', 'Grade' ),
    ( '22-23', 'Requests by School, Grade, Program.xlsx - 2022-23 - Table 1.csv', 'School ID', 'School Name', 'Program Code', 'Program Name', 'Grade' ),
    ( '21-22', 'Enrollment Highlights 2021-22 MR - Program by Rank.xlsx - Table 1.csv', None, 'SchoolName', None, 'Program', 'Grade' ),
    # ( '20-21', '...', None, 'School', None, 'Program', 'Grade' ),
    ( '19-20', 'sfusd_program_requests_by_rank_2019_2020_parsed.csv', None, 'School', None, 'Program', 'Grade' ),
    #( '18-19', 'sfusd_lottery_2018_2019_parsed.csv', 'idSchool', 'SchoolName', None, 'Program', 'Grade' ), # this year is broken
    ( '17-18', 'sfusd_lottery_2017_2018_parsed.csv', 'SchoolCode', 'School', None, 'Prg', 'Grade'),
    ( '16-17', 'sfusd_lottery_2016_2017_parsed.csv', 'SchoolCode', 'School', None, 'Prg', 'Grade'),
    ( '15-16', 'sfusd_lottery_2015_2016_parsed.csv', 'SchoolCode', 'School', None, 'Prg', 'Grade'),
    ( '14-15', 'sfusd_lottery_2014_2015_parsed.csv', 'SchoolCode', 'School', None, 'Prg', 'Grade'),
    ( '13-14', 'sfusd_lottery_2013_2014_parsed.csv', 'SchoolCode', 'School', None, 'Prg', 'Grade'),
    ( '12-13', 'sfusd_lottery_2012_2013_parsed.csv', 'SchoolCode', 'School', None, 'Prg', 'Grade')
]



#
# Check for errors
#
file_content_errors = []

for ay, filename, school_id_column, school_name_column, program_code_column, program_name_column, grade_column in file_descriptions:
    df = pd.read_csv( f'{data_directory}/{filename}' )

    for column in [ school_id_column, school_name_column, program_code_column, program_name_column, grade_column ]:
        if column is None:
            continue

        if column not in df:
            file_content_errors.append( ( ay, filename, column ) )
            print( f'Missing column "{column}" in {filename} for {ay}' )
            print( f'Columns found: {df.columns.tolist()}' )

if len( file_content_errors ) > 0:
    raise Exception( f'Found the following file content errors: {file_content_errors}' )

In [284]:
#
# Parse the files
#
full_df = pd.DataFrame()

def to_bounds ( choice_range ):
    parsed_col = re.sub( r'[^\d\-\+]', '', choice_range )
    parts = re.split( r'\-', parsed_col )
    if len( parts ) > 1:
        return ( int( parts[0] ), int( parts[1] ) )
    elif parts[0][-1] == '+':
        return ( int( parts[0][:-1] ), np.inf )
    else:
        return ( int( parts[0] ), int( parts[0] ) )

for ay, filename, school_id_column, school_name_column, program_code_column, program_name_column, grade_column in file_descriptions:
    df = pd.read_csv( f'{data_directory}/{filename}' )

    choice_columns = [ col for col in df.columns if re.search( r'(?:[Cc]hoice|\+|^\d+)', col ) ]
    choice_columns = { col: to_bounds( col ) for col in choice_columns }

    for choice_column, bounds in choice_columns.items():
        choice_min, choice_max = bounds
        select_columns = [ school_id_column, school_name_column, program_code_column, program_name_column, grade_column, choice_column ]
        select_columns = [ col for col in select_columns if col is not None ]
        subdf = df \
            [select_columns] \
            .rename( columns = {
                school_id_column: 'school_id',
                school_name_column: 'school_name',
                program_code_column: 'program_code',
                program_name_column: 'program_name',
                grade_column: 'grade',
                choice_column: 'num_requests'
            } ) \
            .assign(
                choice_min = choice_min,
                choice_max = choice_max,
                ay = ay
            )
        
        full_df = pd.concat( [ full_df, subdf ], ignore_index = True, axis = 0 )

In [285]:
#
# Do some cleaning
#

In [286]:
#
# Clean grades
# > Note that the "555" may indicate data issues
# > Do we need to do anything about PK or 13?
#
full_df = full_df.assign( grade = full_df.grade.str.upper() )
full_df = full_df[~full_df.grade.isin( [ '555', 'TOTAL' ] )]
full_df = full_df.assign( grade = full_df.grade.fillna( '' ).apply( lambda x: re.sub( r'(?:^0|TH GRADE)', '', x ) ) )
full_df = full_df[full_df.grade != ''] # removes Miraloma Newcomer 2019-2020

display( full_df.grade.unique() )

array(['13', 'TK', 'K', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       '11', '12', 'PK'], dtype=object)

In [287]:
#
# Massage the school IDs
# > Note that there are some inconsistencies in the treatment of ES-associated TK programs
#
modes = [ 'export', 'search', 'import' ]
mode_school_update = 'import'

export_filename = './schools.csv'
import_filename = './schools.csv'

search_school_contains = 'Academy'
search_school_grades = [ '9' ]



#
# Don't mess with the below
#
if mode_school_update == 'export':
    df = full_df[['school_id','school_name']].drop_duplicates()
    df = df.merge( df[~pd.isnull( df.school_id )], on = 'school_name', suffixes = ( '_left', '_right' ), how = 'left' ).drop_duplicates()

    num_missing_ids = len( df[pd.isnull( df.school_id_right )] )
    print( f'Found {num_missing_ids} school[s] without IDs.' )

    if num_missing_ids > 0:
        df.to_csv( export_filename, index = False )

    raise Exception( 'Cannont continue until "import" flag is set.' )



if mode_school_update == 'search':
    df = full_df

    if search_school_contains is not None:
        df = df[df.school_name.str.contains( search_school_contains )]
    if search_school_grades is not None:
        df = df[df.grade.isin( search_school_grades )]

    df = df \
        [['school_name','school_id','grade','ay','num_requests']] \
        .assign( num_requests = lambda _df: _df.num_requests.fillna( 0 ).astype( int ) ) \
        .fillna( '<none>' )
    
    display(
        df
            .groupby( [ col for col in df if col != 'num_requests' ] )
            .sum()
            .reset_index()
            .sort_values( [ 'ay', 'grade' ], ascending = [ False, False ] )
    )

    raise Exception( 'Cannont continue until "import" flag is set.' )



if mode_school_update == 'import':
    school_id_df = pd.read_csv( import_filename )

    if len( school_id_df[pd.isnull( school_id_df.school_id_right )] ) > 0:
        raise Exception( f'Cannot import school IDs; some schools are still missing IDs.' )
    
    if len( school_id_df[~pd.isnull( school_id_df.school_id_left ) & ( school_id_df.school_id_left != school_id_df.school_id_right )] ) > 0:
        raise Exception( f'Cannot import school IDs; some schools have conflicting IDs.' )
    
    school_id_df = school_id_df \
        [['school_id_right','school_name']] \
        .rename( columns = { 'school_id_right': 'school_id' } ) \
        .drop_duplicates()
    
    if len( school_id_df.merge( school_id_df, on = 'school_name' ) ) != len( school_id_df ):
        raise Exception( f'Cannot import school IDs; duplicate school names found.' )
    
    full_df = full_df \
        .merge( school_id_df, on = 'school_name', suffixes = ( '', '_updated' ) ) \
        .assign( school_id = lambda _df: _df.school_id.fillna( _df.school_id_updated ).astype( int ) ) \
        .drop( columns = [ 'school_id_updated' ], axis = 1 ) \

    full_df \
        [['school_id','school_name','ay']] \
        .assign( school_id = lambda _df: _df.school_id.astype( int ) ) \
        .assign( ay_rank = lambda _df: _df[['school_id','ay']].groupby( 'school_id' ).ay.rank( method = 'first', ascending = False ) ) \
        [lambda _df: _df.ay_rank == 1] \
        [['school_id','school_name']] \
        .reset_index( drop = True ) \
        .to_csv( './sfusd-school-id-mapping.csv', index = False )

In [290]:
#
# Massage the program codes
#
# THIS CELL IS ROUGH. It made more sense to do programmatic overwrites.
#
modes = [ 'export', 'search', 'import' ]
mode_program_update = 'import'

search_school_id = 621
search_program_name_contains = None



#
# Some observed programmatic updates
#
full_df = full_df.assign( program_name = lambda _df: _df.program_name.fillna( '<none>' ) )
program_codes = full_df.program_code.unique().tolist()
full_df = full_df.assign( program_code = lambda _df: _df.program_name.map( {
    'at DeAvila ES Chinese Native Speaker': 'CN', # 19-20 error
    'Autism Focus': 'AF', # appears to be the same as MM?
    'Cantonese Immersion (Other languages cohort)': 'CT', # this is a guess
    'Chinese Biliteracy': 'CB', # formally this has become Cantonese Biliteracy
    'Chinese Immersion (Cantonese)': 'CN',
    'Chinese Immersion (Non-Native)': 'CE',
    'Chinese Immersion (Non-English)': 'CT',
    'Chinese Native Speaker': 'CN',
    'Chinese Non-Native Speakers': 'CE',
    'SDC - Deaf and Hard of Hearing': 'DH',
    'Deaf/Hard of Hearing': 'DH',
    'DA': 'AO', # this is a guess
    'DH': 'DH',
    'DT': 'TC', # this is a guess
    'Emotionally Disturbed': 'ED',
    'Filipino Biliteracy': 'FB',
    'Filipino Bilingual': 'FB',
    'FIL': 'FB',
    'Francisco @ McAteer HS Mild Moderate': 'MM', # this is probably a data issue
    'GEN': 'GE',
    'Gen Ed': 'GE',
    'Gen Ed Newcomer': 'GX',
    'General Education': 'GE',
    'General Education (SF International only)': 'GX',
    'HS Mild Moderate': 'MM',
    'Japanese Bilingual': 'JB',
    'Japanese Non-Native Cohort': 'JE',
    'Japanese Bilingual Bicultural - English': 'JE',
    'Japanese Bilingual Bicultural - Japanese': 'JN',
    'Japanese Native Cohort': 'JN',
    'K-8 Spanish Native': 'SN',
    'Korean Immersion (Korean)': 'KN',
    'Korean Immersion (Non-Native)': 'KE',
    'Korean Native Cohort': 'KN',
    'Korean Non-Native Cohort': 'KE',
    'Mandarin Immersion (Mandarin)': 'MN',
    'Mandarin Immersion (Non-Native)': 'ME',
    'Mandarin Native': 'MN',
    'Mandarin Non-Native Cohort': 'ME',
    'Mild Moderate': 'MM',
    'Moderate Severe': 'MS',
    'Chinese Newcomer': 'NC',
    'NEWC': 'NC',
    'Spanish Newcomer': 'NS',
    'NEWS': 'NS',
    'Newcomer Spanish': 'NS',
    'Newcomer': 'NX',
    'NEWX': 'NX',
    'SDC - Auditory/Oral': 'AO',
    'SDC - Social-Emotional/Behavioral Enhanced': 'ED',
    'SDC - Social-Emotional/Behavioral\nEnhanced': 'ED',
    'SDC - Severely Autistic': 'SA',
    'SDC - Moderate to Severe': 'MS',
    'SDC - Mild to Moderate': 'MM',
    'SDC - Mild/Mod Autism Focus': 'MM',
    'Severely Autistic': 'SA',
    'Spanish Biliteracy': 'SB',
    'Spanish Bilingual': 'SB', # changed names to biliteracy
    'Spanish Immersion (Non-Native)': 'SE',
    'Spanish Immersion (Spanish)': 'SN',
    'Spanish Non-Native': 'SE',
    'Spanish Native': 'SN',
    **{ lang: lang for lang in [ 'CHN', 'JPN', 'SPN' ] },
    **{ f'IMM{lang[0]}': f'{lang}IMM' for lang in [ 'CHN', 'KOR', 'MND', 'SPN' ] },
    **{ code: code for code in program_codes },
    **{ f'i{code}': code for code in program_codes }, # this may be an error; pretty sure these are "i" for "integrated"
    **{ f'I{code}': code for code in program_codes } # this may be an error; pretty sure these are "i" for "itegrated"
}).fillna( _df.program_code ))



if mode_program_update == 'export':
    df = full_df[['ay','school_id','program_code','program_name']].drop_duplicates()
    df = df[['school_id','program_code','program_name']][lambda _df: pd.isnull( _df.program_code )].drop_duplicates()
    print( len( df ) )
    display( df.reset_index().tail( 30 ) )



if mode_program_update == 'search':
    df = full_df

    if search_school_id is not None:
        df = df[df.school_id == search_school_id]
    if search_program_name_contains is not None:
        df = df[df.program_name.fillna( '' ).str.contains( search_program_name_contains )]

    df = df[['ay','school_id','program_code','program_name']].drop_duplicates()
    display( df.head( 30 ) )
    display( df.program_name.drop_duplicates() )



if mode_program_update == 'import':
    if len( full_df[pd.isnull( full_df.program_code )] ) > 0:
        raise Exception( 'Cannot continue: some programs have no program code.' )
    
    full_df \
        [['ay','school_id','program_code','program_name']] \
        .drop_duplicates() \
        .to_csv( './sfusd-program-code-mapping.csv', index = False )

In [ ]:
#
# Save everything
#
full_df \
    .assign( num_requests = lambda _df: _df.num_requests.fillna( 0 ).apply( lambda x: re.sub( r'(?:\,|\.\d*$)', '', str( x ) ) ) ) \
    .assign( num_requests = lambda _df: np.where( _df.num_requests.str.len() == 0, 0, _df.num_requests.fillna( 0 ) ).astype( int ) ) \
    .drop( [ 'program_name', 'school_name' ], axis = 1 ) \
    .to_csv( './sfusd-lottery-all.csv', index = False )